<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4_second_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler 


In [6]:
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [7]:
from sklearn.model_selection import train_test_split
# X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
X = df[['sepal_width', 'petal_length']]

scaler_training = StandardScaler() 
scaler_training.fit(X)
scaled_values_training = scaler_training.transform(X)

X = pd.DataFrame(scaled_values_training, columns=[*X.columns.values])
X['target'] = df['target']


In [8]:
X

,sepal_width,petal_length,target
0,1.019004,-1.340227,0.0
1,-0.131979,-1.340227,0.0
2,0.328414,-1.397064,0.0
3,0.098217,-1.283389,0.0
4,1.249201,-1.340227,0.0
...,...,...,...
145,-0.131979,0.819596,2.0
146,-1.282963,0.705921,2.0
147,-0.131979,0.819596,2.0
148,0.788808,0.933271,2.0


In [9]:
from math import exp
from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']
 
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[int(row[-1])] = 1
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
 
# Test training backprop algorithm
seed(68)
dataset = X.values
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 3, n_outputs)
print(network)
train_network(network, dataset, 0.5, 100, n_outputs)
for layer in network:
	print(layer)

[[{'weights': [0.7416689356145733, 0.9408038831472155, 0.7474033968541627]}, {'weights': [0.9081425629614371, 0.11097728168000565, 0.579773181999968]}, {'weights': [0.4318673124107113, 0.26549257874510634, 0.8604091679690017]}], [{'weights': [0.32940149633680194, 0.9736705910655336, 0.36125339691759495, 0.5536247242149994]}, {'weights': [0.7742260752576486, 0.7817480945876376, 0.041785415602753395, 0.7335413506775065]}, {'weights': [0.1060683270342535, 0.3424554918369298, 0.09338908488068065, 0.6053788824313064]}]]
>epoch=0, lrate=0.500, error=61.438
>epoch=1, lrate=0.500, error=57.219
>epoch=2, lrate=0.500, error=44.793
>epoch=3, lrate=0.500, error=40.300
>epoch=4, lrate=0.500, error=38.923
>epoch=5, lrate=0.500, error=38.040
>epoch=6, lrate=0.500, error=37.378
>epoch=7, lrate=0.500, error=36.598
>epoch=8, lrate=0.500, error=35.278
>epoch=9, lrate=0.500, error=33.263
>epoch=10, lrate=0.500, error=30.739
>epoch=11, lrate=0.500, error=27.995
>epoch=12, lrate=0.500, error=25.254
>epoch=1